In [ ]:
# -*- coding: utf-8 -*-
import socket
import threading
import time
import random
from datetime import datetime
from collections import defaultdict

# Configuración de servicios simulados
SERVICIOS = {
    21: {
        "name": "FTP",
        "banner": b"220 Microsoft FTP Service (Version 5.0)\r\n",
        "response": b"230 User logged in.\r\n"
    },
    22: {
        "name": "SSH",
        "banner": b"SSH-2.0-OpenSSH_8.2p1\r\n",
        "response": b""
    },
    80: {
        "name": "HTTP",
        "banner": b"HTTP/1.1 200 OK\r\nServer: Microsoft-IIS/10.0\r\n\r\n",
        "response": b"<html><body><h1>Bienvenido al servidor de prueba</h1></body></html>"
    },
    443: {
        "name": "HTTPS",
        "banner": b"HTTP/1.1 200 OK\r\nServer: nginx/1.18.0\r\n\r\n",
        "response": "<html><body><h1>Conexión segura establecida</h1></body></html>".encode('utf-8')  # tener encuenta la codificacion para texto
    },
    3389: {
        "name": "RDP",
        "banner": b"\x03\x00\x00\x13\x0e\xd0\x00\x00\x124\x00\x02\x1f\x08\x00\x02\x00\x00\x00",
        "response": b""
    },
    445: {
        "name": "SMB",
        "banner": b"\x00\x00\x00\x00\xff\x53\x4d\x42\x72\x00\x00\x00\x00\x88\x01\x00",
        "response": b""
    },
    3306: {
        "name": "MySQL",
        "banner": b"\x4a\x00\x00\x00\x0a\x38\x2e\x30\x2e\x32\x30\x00",
        "response": b""
    }
}

# Puertos cerrados (responden con refused)
CLOSED_PORTS = [135, 139]

# Puertos bloqueados por firewall (no responden)
BLOCKED_PORTS = [23]

# Detección de escaneos
conexiones = defaultdict(int)

def handle_client(conn, addr, port):
    """Maneja la conexión entrante para un puerto específico"""
    global conexiones
    conexiones[addr[0]] += 1

    try:
        # Detección de escaneo (más de 10 conexiones desde la misma IP)
        if conexiones[addr[0]] > 10:
            print(f"[!] Detección de escaneo desde {addr[0]}. Bloqueando...")
            conn.close()
            return

        service = SERVICIOS.get(port)
        if service:
            # Enviar banner
            conn.sendall(service["banner"])

            # Simular interacción básica
            data = conn.recv(1024)
            if data:
                conn.sendall(service["response"])

            # Simular delay aleatorio
            time.sleep(random.uniform(0.1, 0.5))

    except Exception as e:
        print(f"[!] Error en puerto {port}: {str(e)}")
    finally:
        conn.close()

def start_server(port):
    """Inicia un servidor en el puerto especificado"""
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            s.bind(('0.0.0.0', port))
            s.listen(5)
            print(f"[*] Servicio {SERVICIOS[port]['name']} escuchando en puerto {port}")

            while True:
                conn, addr = s.accept()
                print(f"[+] Conexión entrante en puerto {port} desde {addr[0]}")
                threading.Thread(target=handle_client, args=(conn, addr, port)).start()

    except Exception as e:
        print(f"[!] Error al iniciar servicio en puerto {port}: {str(e)}")

def simulate_closed_port(port):
    """Simula un puerto cerrado (respondiendo con connection refused)"""
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            s.bind(('0.0.0.0', port))
            s.listen(5)
            print(f"[*] Puerto {port} configurado como cerrado (responderá con refused)")

            while True:
                conn, addr = s.accept()
                conn.close()  # Cierra inmediatamente para simular refused

    except Exception as e:
        print(f"[!] Error en puerto cerrado {port}: {str(e)}")

def main():
    """Función principal del servidor de simulación"""
    print("""
    **********************************************
    *    SIMULADOR DE SERVIDOR WINDOWS VULNERABLE *
    *        Para demostración por BlackTiger04   *
    **********************************************
    """)

    print("[*] Configurando servicios...")

    # Iniciar servicios abiertos
    for port in SERVICIOS:
        threading.Thread(target=start_server, args=(port,), daemon=True).start()

    # Iniciar puertos cerrados
    for port in CLOSED_PORTS:
        threading.Thread(target=simulate_closed_port, args=(port,), daemon=True).start()

    print("[*] Puertos bloqueados por firewall (sin respuesta):", BLOCKED_PORTS)
    print("[*] Simulación lista. Esperando conexiones...")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n[*] Deteniendo servidor de simulación...")

if __name__ == "__main__":
    main()